In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [2]:
def subject_wise_split(
    x, y, participant, subject_wise=True, test_size=0.10, random_state=42
):
    """Split data into train and test sets via an inter-subject scheme, see:
    Shah, V., Flood, M. W., Grimm, B., & Dixon, P. C. (2022). Generalizability of deep learning models for predicting outdoor irregular walking surfaces.
    Journal of Biomechanics, 139, 111159. https://doi.org/10.1016/j.jbiomech.2022.111159

    Arguments:
        x: nd.array, feature space
        y: nd.array, label class
        participant: nd.array, participant associated with each row in x and y
        subject_wise: bool, choices {True, False}, default = True. True = subject-wise split approach, False random-split
        test_size: float, number between 0 and 1. Default = 0.10. percentage spilt for test set.
        random_state: int. default = 42. Seed selector for numpy random number generator.
    Returns:
        x_train: nd.array, train set for feature space
        x_test: nd.array, test set for feature space
        y_train: nd.array, train set label class
        y_test: nd.array, test set label class
        subject_train: nd.array[string], train set for participants by row of input data
        subjects_test: nd.array[string[, test set for participants by row of input data
    """
    if type(participant) == list:
        participant = np.asarray(participant, dtype=np.float32)

    np.random.seed(random_state)
    if subject_wise:
        # Extract unique participants
        uniq_parti = np.unique(participant)
        # Calculate the number of participants for the test set
        num = np.round(uniq_parti.shape[0] * test_size).astype("int64")
        np.random.shuffle(uniq_parti)
        extract = uniq_parti[0:num]
        test_index = np.array([], dtype="int64")
        for j in extract:
            test_index = np.append(test_index, np.where(participant == j)[0])
        # Remove test set indices from all indices to get train set indices
        train_index = np.delete(np.arange(len(participant)), test_index)
        # Shuffle the train and test indices
        np.random.shuffle(test_index)
        np.random.shuffle(train_index)

    else:
        index = np.arange(len(participant)).astype("int64")
        np.random.shuffle(index)
        num = np.round(participant.shape[0] * test_size).astype("int64")
        # Select train and test set indices
        test_index = index[0:num]
        train_index = index[num:]

    # Extract train and test sets based on the computed indices
    x_train = x[train_index]
    x_test = x[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    subject_train = participant[train_index]
    subject_test = participant[test_index]

    return x_train, x_test, y_train, y_test, subject_train, subject_test

In [3]:
# Specify the correct path to the processed data files
ap = pd.read_csv("path_to_GRF_F_AP_PRO_right.csv")
ml = pd.read_csv("path_to_GRF_F_ML_PRO_right.csv")
v = pd.read_csv("path_to_GRF_F_V_PRO_right.csv")

# Separate the output 'SEX' column
y = ap['SEX']

# Separate the column 'SUBJECT_ID' to be used for splitting
subject = ap['SUBJECT_ID']

# Drop the first four and last 2 columns from each DataFrame
X_data1 = ap.iloc[:, 4:-2]
X_data2 = ml.iloc[:, 4:-2]
X_data3 = v.iloc[:, 4:-2]

In [4]:
# Stack the data to make a tensor
X = np.stack([X_data1.values, X_data2.values, X_data3.values], axis=-1)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test, sub_train, sub_test = subject_wise_split(X, y, subject, subject_wise=True, test_size=0.2, random_state=42)

# One hot encode labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# **CNN model**

In [ ]:
# Initialize empty lists to store the results
results_f1 = []
results_accuracy = []

for i in range(10):
  print("Iteration", i+1)
  # Define the CNN model
  model = Sequential()

  model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(64, kernel_size=3, activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())

  # Dense layers for classification
  model.add(Dense(128, activation='relu'))
  model.add(Dense(2, activation='softmax'))  # 2 is the number of classes

  # Compile the model
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # # Print a summary of the model architecture
  # model.summary()

  y_test_class = np.argmax(y_test, axis=1)

  # Train the model
  history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=False)

  # Generate predictions on the test set
  y_pred = model.predict(X_test)
  y_pred_class = np.argmax(y_pred, axis=1)

  # Calculate classification metrics (e.g., precision, recall, f1-score)
  report = classification_report(y_test_class, y_pred_class, output_dict=True)
  results_f1.append(report["weighted avg"]["f1-score"])
  results_accuracy.append(report["accuracy"])

# Calculate the mean and standard deviation of the f1 score and accuracy
mean_f1 = np.mean(results_f1)
std_f1 = np.std(results_f1)
mean_accuracy = np.mean(results_accuracy)
std_accuracy = np.std(results_accuracy)

# Print results
print("Mean f1 score:", mean_f1)
print("Standard deviation of the f1 score:", std_f1)
print("Mean accuracy:", mean_accuracy)
print("Standard deviation of the accuracy:", std_accuracy)

# **LSTM model**

In [ ]:
# Initialize empty lists to store the results
results_f1 = []
results_accuracy = []

for i in range(10):
  print("Iteration", i+1)
  # Define the LSTM model
  model = Sequential()
  model.add(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
  model.add(Dense(units=2, activation='softmax'))  # Assuming 2 classes for classification

  # Compile the model
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # # Print a summary of the model architecture
  # model.summary()

  y_test_class = np.argmax(y_test, axis=1)

  # Train the model
  history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=False)

  # Generate predictions on the test set
  y_pred = model.predict(X_test)
  y_pred_class = np.argmax(y_pred, axis=1)

  # Calculate classification metrics (e.g., precision, recall, f1-score)
  report = classification_report(y_test_class, y_pred_class, output_dict=True)
  results_f1.append(report["weighted avg"]["f1-score"])
  results_accuracy.append(report["accuracy"])

# Calculate the mean and standard deviation of the f1 score and accuracy
mean_f1 = np.mean(results_f1)
std_f1 = np.std(results_f1)
mean_accuracy = np.mean(results_accuracy)
std_accuracy = np.std(results_accuracy)

# Print results
print("Mean f1 score:", mean_f1)
print("Standard deviation of the f1 score:", std_f1)
print("Mean accuracy:", mean_accuracy)
print("Standard deviation of the accuracy:", std_accuracy)

In [ ]:
# Access training and validation loss from the history object
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

# Create a line plot for training and validation loss
epochs = range(1, len(training_loss) + 1)
plt.plot(epochs, training_loss, 'b', label='Training Loss')
plt.plot(epochs, validation_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()